## Dependencies

In [1]:
import glob
import warnings
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate, Add, Average
from googleqa_utilityscript import *
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/147-googleq-a-train-3fold-bert-base-unc-concat2/' + '*1.h5')
model_path_list += glob.glob('/kaggle/input/147-googleq-a-train-3fold-bert-base-unc-concat2/' + '*2.h5')
model_path_list += glob.glob('/kaggle/input/147-googleq-a-train-3fold-bert-base-unc-concat2/' + '*3.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/147-googleq-a-train-3fold-bert-base-unc-concat2/model_fold_1.h5
/kaggle/input/147-googleq-a-train-3fold-bert-base-unc-concat2/model_fold_2.h5
/kaggle/input/147-googleq-a-train-3fold-bert-base-unc-concat2/model_fold_3.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=True

def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output, pooled_output, hidden_states = bert_model([input_word_ids, input_masks, segment_ids])
    
    h12 = tf.reshape(hidden_states[-1][:, 0],(-1, 1, 768))
    h11 = tf.reshape(hidden_states[-2][:, 0],(-1, 1, 768))
    h10 = tf.reshape(hidden_states[-3][:, 0],(-1, 1, 768))
    h09 = tf.reshape(hidden_states[-4][:, 0],(-1, 1, 768))
    
    x = Concatenate(axis=2)([h12,h11, h10, h09])

    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.950873,0.642491,0.166115,0.577449,0.592081,0.559210,0.652652,0.725086,0.558036,...,0.894715,0.932362,0.557355,0.971735,0.972343,0.833969,0.041788,0.053942,0.935825,0.920255
1,46,0.879622,0.519574,0.005568,0.762051,0.774344,0.921910,0.543394,0.474097,0.083018,...,0.706249,0.950077,0.654613,0.977884,0.980050,0.871318,0.852799,0.113484,0.140753,0.888151
2,70,0.920084,0.670337,0.019830,0.782821,0.866097,0.928456,0.582635,0.545548,0.198648,...,0.866180,0.937141,0.603237,0.975391,0.975906,0.867055,0.082642,0.056313,0.906178,0.913450
3,132,0.901649,0.447407,0.005867,0.742455,0.712146,0.936662,0.546757,0.455162,0.094338,...,0.715246,0.958336,0.672287,0.979629,0.985841,0.909383,0.786492,0.133636,0.564701,0.897873
4,200,0.944559,0.464660,0.029887,0.855558,0.708830,0.875640,0.619402,0.608963,0.117352,...,0.716791,0.922217,0.621175,0.973340,0.970639,0.841437,0.215484,0.098182,0.783774,0.910677


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.908510,0.608350,0.030060,0.740431,0.795054,0.869625,0.578253,0.500882,0.237449,...,0.797208,0.939999,0.660917,0.971475,0.977436,0.882506,0.488320,0.118775,0.563824,0.910033
std,2812.670060,0.033106,0.118618,0.055010,0.087662,0.093269,0.108868,0.050554,0.084551,0.195712,...,0.078207,0.020126,0.043656,0.007944,0.008115,0.034822,0.317408,0.050146,0.279261,0.020758
min,39.000000,0.789054,0.381313,0.003104,0.273896,0.301684,0.315702,0.481455,0.360685,0.016041,...,0.592452,0.847471,0.532471,0.931511,0.937083,0.758823,0.003619,0.010886,0.029063,0.817144
25%,2572.000000,0.887416,0.502195,0.006890,0.691989,0.743862,0.853918,0.542215,0.439268,0.083101,...,0.733117,0.929403,0.631950,0.967256,0.973807,0.863375,0.141963,0.085452,0.330662,0.897762
50%,5093.000000,0.909878,0.600264,0.010611,0.745436,0.799929,0.908592,0.570105,0.475061,0.168867,...,0.800074,0.944861,0.661220,0.972735,0.979114,0.888960,0.564045,0.119868,0.582572,0.913449
75%,7482.000000,0.932059,0.707773,0.024264,0.797405,0.859790,0.933845,0.607100,0.541967,0.352204,...,0.866483,0.954579,0.694030,0.977011,0.983267,0.908200,0.779079,0.152029,0.811435,0.923865
max,9640.000000,0.972530,0.865980,0.559104,0.952243,0.973030,0.963744,0.719966,0.758522,0.860086,...,0.949176,0.978350,0.772695,0.986252,0.991616,0.964569,0.940032,0.277560,0.994849,0.949684
